# Running CNN with Explainers

This file contains the pipeline for training and loading the CNN model and generating explanations for the model.

This model operates on Tensorflow 1.15 and Keras. 

In [1]:
!pip install innvestigate
!pip install aix360
!pip install lime
!pip install Pillow

In [2]:
%tensorflow_version 1.15
import tensorflow
print(tensorflow.__version__)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `1.15`. This will be interpreted as: `1.x`.


TensorFlow 1.x selected.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
import os
import cnn_kuzushiji
import experiment

Using TensorFlow backend.


In [0]:
use_tf = True
if use_tf:
    import tensorflow.keras as keras
else:
    import keras
import tensorflow as tf

In [5]:
os.getcwd()

'/content'

# Train CNN Model
We train the CNN model using the Kuzushiji-49 dataset. Trained models for epochs are saved in `/report/` as .h5 files. 

To run inference without training, change parameter `train_model = False` in `cnn_kuzushiji.experiment()`. In the dictionary `cnn_params`, add in the location of the model file to load as a value of `load_location`.

In [6]:
 #   check GPU
if tf.test.gpu_device_name():
    print("Default GPU Device: {}".format(tf.test.gpu_device_name()))
else:
    print("No GPU.")

num_epochs = 300
batch_size = 256

dl_params = {
        'labels':           ['a', 'me'],
        'label_type':       'int',
        'presorted':        True,
        'file_locs':        ["./data/kuzushiji-49/0", "./data/kuzushiji-49/33"],
        'file_exten':       '.png',
        #'set_ratio':        [0.7, 0.15, 0.15],
        'set_ratio':        [0.8, 0.2],
        'batch_size':       batch_size,
        'target_size':      (28,28),
        'balanced':         True,
        'grayscale':        True,
        'superspeedmode':   False                #   trades off memory efficiency for less computation (USE AT YOUR OWN RISK)
}

train_params = {
        'learning_rate':        0.0075,
        'learning_decay':       0.9,
        'num_epochs':           num_epochs,
        'batch_size':           batch_size,
        'save_n_epoch':         10,
        'loss_fcn':             'binary_crossentropy',
        'report_metrics':       ['accuracy'],
        'plot_filename':        'training_plot',
        'test_filename':        'test_report',
        'report_dir':           './report/',
        }


img_height, img_width, img_channels = (28, 28, 3)
num_classes = 1             #   use 1 if binary

cnn_params = {
        'input_size':           (img_height, img_width, img_channels),
        'num_filters':          3,
        'filter_size':          (5, 5),
        'stride':               1,
        'padding':              'same',
        'conv_activation':      'relu',
        'pool_size':            (2,2),
        'dropout':              0.1,
        'num_conv_layers':      2,
        'output_dim':           num_classes,
        'fc_activation':        'sigmoid',
        'load_location':        '',
        'report_metrics':       ['accuracy'],
        }

No GPU.


In [7]:
cnn_kuzushiji.experiment(dl_params, cnn_params, train_params, train_model=True)




Saving parameters...
Loading data...
Loading data from ./data/kuzushiji-49/0 ...
Getting samples for class 0...
There are 2715 training, 678validation, 552 test samples.
Loading data from ./data/kuzushiji-49/33 ...
Getting samples for class 1...
There are 2715 training, 678validation, 552 test samples.
There are 5430 training samples and 22 training batches.
There are 1356 validation samples and 6 validation batches.
There are 1104 test samples and 5 test batches.
Converting from grayscale to RGB...
Building classifier...





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 28, 28, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 3)         228       
___________________

# Generate Explainers
We generate 3 types of explanations for the trained model:
- LIME
- Prototypes
- Saliency maps

The original model used for the study is `/model/model_e299.h5`.  To use a different trained model, change the value of `load_location` in the dictionary `model_params`. 

Outputted explanations are stored in `/output/`.

In [8]:
num_epochs = 300
batch_size = 256

dl_params = {
        'labels':           ['sun', 'moon'],
        'label_type':       'int',
        'presorted':        True,
        'file_locs':        ["./data/kuzushiji-49/0", "./data/kuzushiji-49/33"],
        'file_exten':       '.png',
        'set_ratio':        [0.8, 0.2],
        'batch_size':       batch_size,
        'target_size':      (28,28),
        'balanced':         True,
        'grayscale':        True,
        'superspeedmode':   False                #   trades off memory efficiency for less computation (USE AT YOUR OWN RISK)
}

model_params = {
    'output_dim':           1,
    'activation':           'relu',
    'load_location':        './model/model_e299.h5'
}

#   dictionary containing each explainer as key and list of list of text, with each
#   list within a list contains text for each explanation with an explainer

text_files = ['lime.txt', 'heatmap.txt', 'prototypes.txt']
text_dict = {}
for file in text_files:
    text_dict = experiment.populate_text_dict(file, text_dict)

print(text_dict['prototypes'])

explain_params = {
            'border' :          (0, 0),
            'num_images':       3,
            'font_file' :       'Arial.ttf',
            'text_size' :       30,
            'text_dict':        text_dict,
            'imagenet':         False
            }
    
#   create save directory
save_dir = "./output/"

[["WALTER : I think the images on the left represent the sun.  The smaller images on the right are the 3 most similar images that I've seen before, when compared to each of these images.  The ground-truth labels of these images are below each one.", ' ', ' ', "WALTER : OK, I just checked the labels for the images on the left.  I'm correct :)  They all represent the sun!"], ["WALTER : I think the images on the left represent the moon.  The smaller images on the right are the 3 most similar images that I've seen before, when compared to each of these images.  The ground-truth labels of these images are below each one.", ' ', ' ', "WALTER : OK, I just checked the labels for the images on the left.  I'm correct :)  They all represent the moon!"], ["WALTER : I think the images on the left represent the moon.  The smaller images on the right are the 3 most similar images that I've seen before, when compared to each of these images.  The ground-truth labels of these images are below each one.

In [11]:
experiment.experiment(dl_params, model_params, explain_params, save_dir)

Loading dataset...
Loading data from ./data/kuzushiji-49/0 ...
Getting samples for class 0...
There are 2715 training, 678validation, 552 test samples.
Loading data from ./data/kuzushiji-49/33 ...
Getting samples for class 1...
There are 2715 training, 678validation, 552 test samples.
There are 5430 training samples and 22 training batches.
There are 1356 validation samples and 6 validation batches.
There are 1104 test samples and 5 test batches.
X_train.shape:  (5430, 28, 28, 1)
Converting from grayscale to RGB...
Flattening data...
Finding prototypes for only label 0
Explaining...


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Plotting prototypes...
Number of prototypes:  (5, 28, 28, 3)
Flattening data...
Finding prototypes for only label 1
Explaining...


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Plotting prototypes...
Number of prototypes:  (5, 28, 28, 3)
Loading pre-existing classifier...


TypeError: ignored